In [12]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/NIOMATA/strollerGAN_austria/app/ganspace_api/ganspace
!pip install pyrebase4
!pip install boto3
!pip install fbpca
!pip install flask_cors
!pip install flask_ngrok

/content/drive/MyDrive/NIOMATA/strollerGAN_austria/app/ganspace_api/ganspace
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 460kB 27.6MB/s 
     |████████████████████████████████| 1.9MB 57.0MB/s 
     |████████████████████████████████| 81kB 12.4MB/s 
     |████████████████████████████████| 3.2MB 54.7MB/s 
  Created wheel for gcloud: filename=gcloud-0.18.3-cp37-none-any.whl size=602938 sha256=7f66bbd165cb31c11224db25b3cf6bf94531503c8d626940b9993d43ed30176e
  Stored in directory: /root/.cache/pip/wheels/b9/9b/9c/a01be401658fea33b93a35d03921b0c638266821b264dc8662
Successfully built gcloud
     |████████████████████████████████| 102kB 8.3MB/s 
     |████████████████████████████████| 7.4MB 22.2MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 153kB 55.0MB/s 
  Created wheel for boto3: filename=boto3-1.17.48-py2.py3-none-any.whl size=128779 sha256=7aff0bead32eeac755fa4b9fd105419d408632cb7761d8e2

# **import everything**

In [3]:
import os
import torch
import random
import tqdm
import flask
import json

import numpy as np

from generator import Model
from PIL import Image

from flask_cors import CORS
from flask_ngrok import run_with_ngrok 
from flask import request, Response, send_file
from rules.stroller_dict import get_stroller_dict
from rules.childseat_dict import get_childseat_dict

StyleGAN2: Optimized CUDA op FusedLeakyReLU not available, using native PyTorch fallback.
StyleGAN2: Optimized CUDA op UpFirDn2d not available, using native PyTorch fallback.


In [4]:
def generate_seed(num):
    return [random.randint(0,9999) for i in range(num)]

def generate_combination(start,end):
    res = []
    for i in range(start,end) :
        for j in range(i,end+1):
            if i == j:
                continue
            res.append((i,j))
    
    return res

def generate_image_to_API(output_dir,model):
    
    seeds = generate_seed(6)
    combinations  = generate_combination(1,6)
    
    model.generate_transversal(output_dir=output_dir,
                               combinations=combinations,
                               num=seeds)
    
def generate_image_style(model,step,scale,ls,le,z):
    z = model.generate_z(1)
    z = model.normalize_z(z)
    res = []
    
    for i in range(-5,5):
        z_styles = apply_rule(z,model.model,i,scale,ls,le)
        img = model.model.sample_np(z_styles)
        img = Image.fromarray((img * 255).astype(np.uint8)).resize((500,500),Image.LANCZOS)
        res.append(img)
        
    return res
    
def apply_rule(z,model,step,scale,ls,le):
    rule_path ='rules/4W_to_3W.npy'
    rule = np.load(rule_path)
    
    z = [z]*model.get_max_latents()

    for l in range(ls, le):
        z[l] = z[l] + rule * step * scale
    
    return z

# **INIT**

In [5]:
childseat = Model('childseat')
stroller = Model('stroller')
pushchair = Model('pushchair')

the shape of the latent is torch.Size([512])
the shape of the latent is torch.Size([512])
the shape of the latent is torch.Size([512])


In [6]:
stroller_rules = get_stroller_dict()
childseat_rules = get_childseat_dict()

In [7]:
output_dir = '/content'

In [8]:
app = flask.Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route('/', methods=['GET'])
def home():
    return "<h1>HELLO!</h1><p>API to send array of images</p>"

@app.route('/generate_image',methods=['GET','POST']) 
def generate_image():
    cat = request.args.get('category')
    
    if cat == 'childseat':
        generate_image_to_API(output_dir,childseat)
    if cat == 'stroller':
        generate_image_to_API(output_dir,stroller)
    if cat == 'pushchair':
        generate_image_to_API(output_dir,pushchair)
    return json.dumps({'success':True}), 200, {'ContentType':'application/json'} 

@app.route('/generate_style',methods=['GET','POST']) 
def generate_style():
    cat = request.args.get('category')
    filename = request.args.get('filename')
    style_name = request.args.get('name')
    
    z = np.load(os.path.join(output_dir,cat,filename))

    if cat == 'childseat':
        data = next((x for x in childseat_rules if x['name'] == style_name), None)
        childseat.generate_style(z,data,'output')
    if cat == 'stroller':
        data = next((x for x in stroller_rules if x['name'] == style_name), None)
        stroller.generate_style(z,data,'output')
    if cat == 'pushchair':
        data = next((x for x in stroller_rules if x['name'] == style_name), None)
        pushchair.generate_style(z,data,'output')

    return json.dumps({'success':True}), 200, {'ContentType':'application/json'}

In [9]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://fbf9c6ae160c.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [09/Apr/2021 06:51:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/Apr/2021 06:51:52] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [09/Apr/2021 06:51:53] "GET / HTTP/1.1" 200 -


In [11]:
generate_image_to_API(output_dir,childseat)

FileNotFoundError: ignored